In [ ]:
!pip install gdown

In [ ]:
!gdown https://drive.google.com/file/d/1c-J0AlAVMKVYtriwlxqkbx94gafAQLjB/view?usp=sharing --fuzzy

Downloading...
From (original): https://drive.google.com/uc?id=1c-J0AlAVMKVYtriwlxqkbx94gafAQLjB
From (redirected): https://drive.google.com/uc?id=1c-J0AlAVMKVYtriwlxqkbx94gafAQLjB&confirm=t&uuid=8f3aed9e-34d7-48e4-b85a-876476283d45
To: /content/driver_161_90frame.tar.gz
100% 5.04G/5.04G [01:03<00:00, 79.0MB/s]


In [ ]:
!tar -xvzf driver_161_90frame.tar.gz

Streaming output truncated to the last 5000 lines.
driver_161_90frame/06032332_0990.MP4/02880.lines.txt
driver_161_90frame/06040114_1024.MP4/02700.jpg
driver_161_90frame/06040111_1023.MP4/02070.lines.txt
driver_161_90frame/06032113_0944.MP4/02700.lines.txt
driver_161_90frame/06040214_1044.MP4/01440.jpg
driver_161_90frame/06040314_1064.MP4/00180.jpg
driver_161_90frame/06031608_0865.MP4/00090.jpg
driver_161_90frame/06031625_0871.MP4/01620.jpg
driver_161_90frame/06031152_0826.MP4/00990.jpg
driver_161_90frame/06031725_0891.MP4/00360.jpg
driver_161_90frame/06032252_0977.MP4/02430.jpg
driver_161_90frame/06032231_0970.MP4/03060.jpg
driver_161_90frame/06040053_1017.MP4/03240.jpg
driver_161_90frame/06032329_0989.MP4/03870.lines.txt
driver_161_90frame/06030819_0755.MP4/05310.lines.txt
driver_161_90frame/06031743_0897.MP4/00900.lines.txt
driver_161_90frame/06031323_0849.MP4/02880.jpg
driver_161_90frame/06031819_0909.MP4/02610.lines.txt
driver_161_90frame/06031228_0838.MP4/02250.lines.txt
driver_1

In [ ]:
import os
import pickle
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
os.listdir('drive/MyDrive/LaneDetection') #Check if you have all the files

['CUDatamap.pkl',
 'CUDatamap_stline2.pkl',
 'CUDatamap_line2.pkl',
 'CUDatamap_coef.pkl',
 'vgg16.pt',
 'model_confidence.pt',
 '.ipynb_checkpoints',
 'model_1.pt']

In [ ]:
with open("drive/MyDrive/LaneDetection/CUDatamap_line2.pkl","rb") as f:
    data_line_map = pickle.load(f)
with open("drive/MyDrive/LaneDetection/CUDatamap_stline2.pkl","rb") as f:
    data_stline_map = pickle.load(f)

In [ ]:
y_col = np.linspace(400,580,19)

class LaneDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_line_map,data_stline_map,scale=None,reg_scale=10):

        self.new_data_keys = list(data_line_map.keys())
        self.data_line_map = data_line_map
        self.data_stline_map = data_stline_map
        self.scale = scale
        image_loc = self.new_data_keys[0]
        image = cv2.imread(image_loc)
        height,width,channel = image.shape
        self.height = height
        self.width = width
        self.reg_scale = reg_scale




    def __len__(self):
        return len(self.new_data_keys)




    def get_lane_angle(self,myline):
        global y_col #its actually row
        row = y_col[-1]
        col = myline[-1]

        new_col = col - self.width//2
        new_row = row + self.height

        angle = -np.arctan2(new_col,new_row)
        # if angle<0:
        #     angle = 2*np.pi+angle

        return angle

    def get_lane_bins(self,stline):
        angle = self.get_lane_angle(stline)
        angle = np.clip(angle,-1,1)
        mybin = int(round(angle/0.125))+8 #17 bins -1 to 1 as 0 to 17

        return mybin,angle


    def get_regression_confidence_mat(self,stlines,lines):
        regression_mat = np.zeros((19,17),dtype=np.float32) #19 rows and 17 for 17 bins
        confidence_mat = np.zeros((17,),dtype=np.float32)
        for stline,line in zip(stlines,lines):
            mybin,angle = self.get_lane_bins(stline=stline)
            regression_mat[:,mybin] = line
            confidence_mat[mybin] = 1
        return regression_mat.flatten(),confidence_mat

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image_loc = self.new_data_keys[idx]
        lines = self.data_line_map[image_loc] #Since
        image = cv2.imread(image_loc)

        if self.scale is not None:
            image = cv2.resize(image,self.scale)

        stlines = self.data_stline_map[image_loc]


        #visualize_bundle = (image,lines,stlines,idx) #use only for visualization


        image_tensor = torch.tensor(image,dtype=torch.float32)/255 #convert to tensor and normalise
        image_tensor2 = image_tensor.permute(2,0,1) #bring the channel to front

        regression_mat,confidence_mat  = self.get_regression_confidence_mat(stlines,lines)
        regression_mat_tensor = torch.tensor(regression_mat,dtype=torch.float32)
        regression_mat_tensor = torch.clip(regression_mat_tensor,-1500,2500)/self.reg_scale #Clip and also reduce the scale
        confidence_mat_tensor = torch.tensor(confidence_mat,dtype=torch.float32)

        #print(image_loc,sum(regression_mat))


        return image_tensor2,regression_mat_tensor,confidence_mat_tensor

In [ ]:
import torch.nn as nn
import torchvision.models as models
from torch.nn import functional as F

In [ ]:
vgg16 = models.vgg16(pretrained=True)
for param in vgg16.parameters():
            param.requires_grad = False #Disable the vgg trainings

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 98.8MB/s]


In [ ]:
class VGGBlock(nn.Module):
    def __init__(self):
        super(VGGBlock, self).__init__()
        self.block1 = nn.Sequential(*list(vgg16.features.children())[:5])
        self.block1t4 = nn.Sequential(*list(vgg16.features.children())[5:16])  # Block 4 of VGG16

    def forward(self, x):
      x1_b = self.block1(x)  # Output will be of shape (b, 64, 112, 112)
      x2 = self.block1t4(x1_b)  # Output will be of shape (b, 256, 56, 56)
      return x1_b,x2


class UpSampleNeck(nn.Module):
    def __init__(self):
        super(UpSampleNeck, self).__init__()
        # Trainable 3x3 convolution to reduce features from 64 to 32
        self.conv3x3_1 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)  # BatchNorm after first conv

        # Reduce Block 4 features from 256 to 128
        self.reduce_conv_block4 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)  # BatchNorm after Block 4 reduction

        # Upsample
        self.upconv = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(64)  # BatchNorm after upconv

        # Convolution layers after concatenation
        self.conv3x3_2 = nn.Conv2d(in_channels=96, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(128)  # BatchNorm after concat
        self.avg_pool_2 = nn.AvgPool2d(kernel_size=2, stride=2)


    def forward(self,x1_b,x2):
        # Pass through the 3x3 convolution layer, batch norm, and ReLU
        x1 = self.conv3x3_1(x1_b)  # Output will be of shape (b, 32, 112, 112)
        x1 = self.bn1(x1)
        x1 = F.relu(x1)

        # Block 4 forward pass (without max-pooling)
        x2 = self.reduce_conv_block4(x2)
        x2 = self.bn2(x2)
        x2 = self.upconv(x2)  # Output will be of shape (b, 64, 112, 112)
        x2 = self.bn3(x2)
        x2 = F.relu(x2)

        # Concatenate x1 and x2 along the channel dimension
        x_concat = torch.cat((x1, x2), dim=1)  # Output will be of shape (b, 96, 112, 112)

        # Convolution layer after concatenation
        x3 = self.conv3x3_2(x_concat)
        x3 = self.bn4(x3)
        x3 = self.avg_pool_2(x3)  # Output will be of shape (b, 128, 56, 56)

        x5 = F.relu(x3) #Will be also used for confidence calculation

        return x5,x_concat


class LaneConfidenceHead(nn.Module):
    def __init__(self):
        super(LaneConfidenceHead, self).__init__()

        self.conv3x3_9 = nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=1)
        self.bn11 = nn.BatchNorm2d(256)
        self.maxpool_5 = nn.MaxPool2d(kernel_size=3,stride=3)

        self.conv3x3_10 = nn.Conv2d(in_channels=256,out_channels=64,kernel_size=1,stride=1)
        self.bn12 = nn.BatchNorm2d(64)
        #self.maxpool_6 = nn.MaxPool2d(kernel_size=3,stride=3)

        self.conv3x3_11 = nn.Conv2d(in_channels=64,out_channels=8,kernel_size=1,stride=1)
        self.bn13 = nn.BatchNorm2d(8)
        #self.maxpool_7 = nn.MaxPool2d(kernel_size=3,stride=3)




        self.Linear_c1 = nn.Linear(in_features=2592,out_features=256)
        self.Linear_c2 = nn.Linear(in_features=256,out_features=17)

        self.flatten = nn.Flatten()


    def forward(self,x5):


      x5 = self.conv3x3_9(x5)
      x5 = self.bn11(x5)
      x5 = self.maxpool_5(x5)
      x5 = F.relu(x5)



      x5 = self.conv3x3_10(x5)
      x5 = self.bn12(x5)
      #x5 = self.maxpool_6(x5)
      x5 = F.relu(x5)



      x5 = self.conv3x3_11(x5)
      x5 = self.bn13(x5)

      #x5 = self.maxpool_7(x5)
      x5 = F.relu(x5)

      x5 = self.flatten(x5)



      x5 = self.Linear_c1(x5)
      x5 = F.sigmoid(x5)
      x5 = self.Linear_c2(x5)


      return x5

# Add positional encoding
class PositionalEncoding(nn.Module):
    def __init__(self,device, d_model, max_len=256):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)  # Add batch dimension
        self.encoding = self.encoding.to(device)




    def forward(self, x):

        return x + self.encoding[:, :x.size(1)]


class Regression_Head(nn.Module):
    def __init__(self,device):
        super(Regression_Head, self).__init__()
        self.conv3x3_12 = nn.Conv2d(in_channels=96,out_channels=12,kernel_size=1,stride=1,padding=1)
        # Patch extraction using unfold
        self.patch_size = 7
        self.unfold = nn.Unfold(kernel_size=self.patch_size, stride=self.patch_size)
        self.fold = nn.Fold(output_size=(112, 112), kernel_size=self.patch_size, stride=self.patch_size)
        num_patches = 256  # Number of patches
        embedding_dim = 588  # Each patch dimension
        channels = 12
        self.patch_embedding_layer = nn.Linear(embedding_dim, embedding_dim)

        self.positional_encoding = PositionalEncoding(device,embedding_dim, max_len=num_patches)

        # Define multi-head attention layer
        num_heads = 2
        self.attention_layer = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads, dropout=0.1)


        self.Conv2d_13 = nn.Conv2d(in_channels=12,out_channels=6,kernel_size=2,stride=2)
        self.bn_14 = nn.BatchNorm2d(6)
        self.Conv2d_15 = nn.Conv2d(in_channels=6,out_channels=2,kernel_size=2,stride=2)
        self.bn_16 = nn.BatchNorm2d(2)

        self.flatten = nn.Flatten()
        self.linear_final = nn.Linear(in_features=1568,out_features=512)
        self.linear_final2 = nn.Linear(in_features=512,out_features=19*17)




    def forward(self,x_concat):
        x6 = self.conv3x3_12(x_concat)
        x6 = F.relu(x6)

        patches = self.unfold(x6) # Shape: [batch_size,channels * patch_size * patch_size, num_patches]
        #batch_size = patches.size(0)
        #channels = patches.size(1)
        #print(patches.shape)
        patches = patches.permute(0, 2,1)

        patches = self.patch_embedding_layer(patches)
        #print("After Patch Embedding",patches.shape)
        patches = self.positional_encoding(patches)
        #print("After Patch Embedding",patches.shape)

        # Apply multi-head attention
        attention_output, _ = self.attention_layer(patches, patches, patches)
        #print("After Attention",attention_output.shape)

        # Fold the attention output back to the original image size
        attention_output = attention_output.permute(0, 2, 1)
        transformed_feature_map = self.fold(attention_output)
        #print("After Fold",transformed_feature_map.shape)

        conv_out = self.Conv2d_13(transformed_feature_map)
        conv_out = self.bn_14(conv_out)
        cov_out = F.relu(conv_out)

        conv_out = self.Conv2d_15(cov_out)
        conv_out = self.bn_16(conv_out)
        conv_out = F.relu(conv_out)

        conv_out = self.flatten(conv_out)
        conv_out = self.linear_final(conv_out)
        conv_out = F.relu(conv_out)
        conv_out = self.linear_final2(conv_out)

        return conv_out




In [ ]:
class LaneDetection_2(nn.Module):
    def __init__(self,device):
        super(LaneDetection_2, self).__init__()

        self.vggblock = VGGBlock()
        self.upsample_neck = UpSampleNeck()
        self.laneconfidence_head = LaneConfidenceHead()

        self.regression_head = Regression_Head(device)





    def forward(self, x):
        # Block 1 forward pass
        x1_b,x2 = self.vggblock(x)

        x5,x_concat = self.upsample_neck(x1_b,x2)

        #print(x5.shape,x_concat.shape)

        x6 = self.laneconfidence_head(x5)

        x7 = self.regression_head(x_concat)

        return x6,x7


In [ ]:
class LaneLoss(nn.Module):
    def __init__(self):
        super(LaneLoss, self).__init__()
        self.regression_loss_fn = nn.MSELoss(reduction='none')  # Use 'none' to calculate loss per sample
        self.confidence_loss_fn = nn.BCEWithLogitsLoss()  # Applies a logistic loss to the confidence scores

    # def forward(self, regression_pred, regression_target, confidence_pred, confidence_target): #Ignore this for now
    #     # Regression loss where confidence is 1
    #     regression_pred = regression_pred.view(-1, 19, 17)
    #     regression_target = regression_target.view(-1, 19, 17)

    #     #print(regression_pred.shape)

    #     # Apply confidence target to each column of the regression output
    #     confidence_target_expanded = confidence_target.unsqueeze(1).expand_as(regression_pred)
    #     #print(confidence_target_expanded)
    #     regression_loss = self.regression_loss_fn(regression_pred, regression_target)  # Shape: [batch_size, 19, 17]
    #     regression_loss = (regression_loss * confidence_target_expanded).mean()

    #     # Confidence loss (logistic loss)
    #     confidence_loss = self.confidence_loss_fn(confidence_pred, confidence_target)

    #     # Combine the losses
    #     total_loss = regression_loss + confidence_loss

    #     return total_loss


    def forward(self, regression_pred, regression_target, confidence_pred, confidence_target):
        # # Regression loss where confidence is 1
        # regression_pred = regression_pred.view(-1, 19, 17)
        # regression_target = regression_target.view(-1, 19, 17)

        # #print(regression_pred.shape)

        # # Apply confidence target to each column of the regression output
        # confidence_target_expanded = confidence_target.unsqueeze(1).expand_as(regression_pred)
        # #print(confidence_target_expanded)
        # regression_loss = self.regression_loss_fn(regression_pred, regression_target)  # Shape: [batch_size, 19, 17]
        # regression_loss = (regression_loss * confidence_target_expanded).mean()

        # Confidence loss (logistic loss)
        confidence_loss = self.confidence_loss_fn(confidence_pred, confidence_target)

        # Combine the losses
        total_loss = confidence_loss

        return total_loss

class LaneCondidenceLoss(nn.Module):
    def __init__(self):
        super(LaneCondidenceLoss, self).__init__()
        self.confidence_loss_fn = nn.BCEWithLogitsLoss()  # Applies a logistic loss to the confidence scores

    def forward(self, confidence_pred, confidence_target):
        confidence_loss = self.confidence_loss_fn(confidence_pred, confidence_target)
        return confidence_loss

#### Training starts below

In [ ]:
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LaneDetection_2(device)

model = model.to(device)
loss_fn = LaneCondidenceLoss()
lane_dataset = LaneDataset(data_line_map=data_line_map,data_stline_map=data_stline_map,scale=(224,224))
dataloader = DataLoader(lane_dataset, batch_size=32,shuffle=True)
# Initialize the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10  # Adjust the number of epochs as needed



In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters in million: {total_params/1000000}")

Number of parameters in million: 5.974732


In [ ]:
device

device(type='cuda')

In [ ]:
for param in model.vggblock.parameters():
            param.requires_grad = False #Disable the vgg trainings

for param in model.upsample_neck.parameters():
            param.requires_grad = True #Enable for neck

for param in model.laneconfidence_head.parameters():
            param.requires_grad = True #Enable for neck

for param in model.regression_head.parameters():
            param.requires_grad = False #Disable the regression head for now as we are not going to train it


In [ ]:
import time

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    #print("Start",time.time())
    for batch_idx, (images, regression_targets, confidence_targets) in enumerate(dataloader):
        images = images.to(device)
        # max_lane_dis = max(max_lane_dis,regression_targets.max())
        # min_lane_dis = min(min_lane_dis,regression_targets.min())
        #print("Data fetched",time.time()
        regression_targets = regression_targets.to(device)
        confidence_targets = confidence_targets.to(device)
        #print("Datafetched",time.time())
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass

        confidence_pred,regression_pred = model(images)

        # Calculate loss
        loss = loss_fn(confidence_pred, confidence_targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Update running loss
        running_loss += loss.item()

        # Print progress
        if batch_idx % 20 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}')
            torch.save(model,"drive/MyDrive/LaneDetection/model_1.pt")
        #print("Model stepped",time.time())
    # Print epoch loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

print("Training completed.")

Epoch [1/10], Batch [1/507], Loss: 0.6704
Epoch [1/10], Batch [21/507], Loss: 0.3751
Epoch [1/10], Batch [41/507], Loss: 0.3523
Epoch [1/10], Batch [61/507], Loss: 0.3427
Epoch [1/10], Batch [81/507], Loss: 0.3305
Epoch [1/10], Batch [101/507], Loss: 0.3092
Epoch [1/10], Batch [121/507], Loss: 0.3041
Epoch [1/10], Batch [141/507], Loss: 0.3204
Epoch [1/10], Batch [161/507], Loss: 0.2876
Epoch [1/10], Batch [181/507], Loss: 0.2518
Epoch [1/10], Batch [201/507], Loss: 0.2299
Epoch [1/10], Batch [221/507], Loss: 0.2212
Epoch [1/10], Batch [241/507], Loss: 0.2578
Epoch [1/10], Batch [261/507], Loss: 0.2380
Epoch [1/10], Batch [281/507], Loss: 0.2076
Epoch [1/10], Batch [301/507], Loss: 0.2102
Epoch [1/10], Batch [321/507], Loss: 0.2011
Epoch [1/10], Batch [341/507], Loss: 0.1842
Epoch [1/10], Batch [361/507], Loss: 0.2333
Epoch [1/10], Batch [381/507], Loss: 0.2708
Epoch [1/10], Batch [401/507], Loss: 0.2341
Epoch [1/10], Batch [421/507], Loss: 0.1999
Epoch [1/10], Batch [441/507], Loss: 0

In [ ]:
torch.save(vgg16,"drive/MyDrive/LaneDetection/vgg16.pt")

In [ ]:
class LaneRegressionLoss(nn.Module):
    def __init__(self):
        super(LaneRegressionLoss, self).__init__()
        self.regression_loss_fn = nn.MSELoss(reduction='none')  # Use 'none' to calculate loss per sample
        self.confidence_loss_fn = nn.BCEWithLogitsLoss()  # Applies a logistic loss to the confidence scores

    def forward(self, regression_pred, regression_target,confidence_target):
        # Regression loss where confidence is 1
        regression_pred = regression_pred.view(-1, 19, 17)
        regression_target = regression_target.view(-1, 19, 17)

        # Apply confidence target to each column of the regression output
        confidence_target_expanded = confidence_target.unsqueeze(1).expand_as(regression_pred)
        #print(confidence_target_expanded)
        regression_loss = self.regression_loss_fn(regression_pred, regression_target)  # Shape: [batch_size, 19, 17]
        regression_loss = (regression_loss * confidence_target_expanded).mean()


        return regression_loss


In [ ]:
# torch.save(model,"drive/MyDrive/LaneDetection/LaneViT_confidence.pt") #This is only trained for confidence
# torch.save(model.state_dict(),"drive/MyDrive/LaneDetection/LaneViT_confidence_state_dict.pt") #Save the parameters also. Remember vgg16 parameters are seperately needed here

In [ ]:
model = torch.load("drive/MyDrive/LaneDetection/LaneViT_confidence.pt")

<ipython-input-44-580f3650455b>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/LaneDetection/LaneViT_confidence.pt")


In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters in million: {total_params/1000000}")

Number of parameters in million: 5.974732


In [ ]:
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
loss_fn = LaneRegressionLoss()
lane_dataset = LaneDataset(data_line_map=data_line_map,data_stline_map=data_stline_map,scale=(224,224))
dataloader = DataLoader(lane_dataset, batch_size=32,shuffle=True)
# Initialize the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10  # Adjust the number of epochs as needed



In [ ]:
for param in model.vggblock.parameters():
            param.requires_grad = False #Disable the vgg trainings

for param in model.upsample_neck.parameters():
            param.requires_grad = False #Neck is already trained

for param in model.laneconfidence_head.parameters():
            param.requires_grad = False #Confidence is already trained

for param in model.regression_head.parameters():
            param.requires_grad = True #Now train only for regression head


In [ ]:
import time

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    #print("Start",time.time())
    for batch_idx, (images, regression_targets, confidence_targets) in enumerate(dataloader):
        images = images.to(device)
        # max_lane_dis = max(max_lane_dis,regression_targets.max())
        # min_lane_dis = min(min_lane_dis,regression_targets.min())
        #print("Data fetched",time.time()
        regression_targets = regression_targets.to(device)
        confidence_targets = confidence_targets.to(device)
        #print("Datafetched",time.time())
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass

        confidence_pred,regression_pred = model(images)

        # Calculate loss
        loss = loss_fn(regression_pred, regression_targets,confidence_targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Update running loss
        running_loss += loss.item()

        # Print progress
        if batch_idx % 20 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}')
            torch.save(model,"drive/MyDrive/LaneDetection/model_reg.pt")
        #print("Model stepped",time.time())
    # Print epoch loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

print("Training completed.")

Epoch [1/10], Batch [1/507], Loss: 1359.3539
Epoch [1/10], Batch [21/507], Loss: 239.0348
Epoch [1/10], Batch [41/507], Loss: 19.5021
Epoch [1/10], Batch [61/507], Loss: 12.9859
Epoch [1/10], Batch [81/507], Loss: 11.0409
Epoch [1/10], Batch [101/507], Loss: 1.7147
Epoch [1/10], Batch [121/507], Loss: 3.1006
Epoch [1/10], Batch [141/507], Loss: 2.5716
Epoch [1/10], Batch [161/507], Loss: 2.5632
Epoch [1/10], Batch [181/507], Loss: 2.0368
Epoch [1/10], Batch [201/507], Loss: 11.5482
Epoch [1/10], Batch [221/507], Loss: 6.9254
Epoch [1/10], Batch [241/507], Loss: 5.4316
Epoch [1/10], Batch [261/507], Loss: 2.9548
Epoch [1/10], Batch [281/507], Loss: 3.5509
Epoch [1/10], Batch [301/507], Loss: 2.7841
Epoch [1/10], Batch [321/507], Loss: 1.9634
Epoch [1/10], Batch [341/507], Loss: 3.4766
Epoch [1/10], Batch [361/507], Loss: 95.3285
Epoch [1/10], Batch [381/507], Loss: 4.4498
Epoch [1/10], Batch [401/507], Loss: 5.2660
Epoch [1/10], Batch [421/507], Loss: 5.7654
Epoch [1/10], Batch [441/507

In [ ]:
#The model still shows signs of improvement. You can continue it when u have time.

In [ ]:
# torch.save(model,"drive/MyDrive/LaneDetection/LaneViT_regression_2.pt") #This is only trained for confidence
# torch.save(model.state_dict(),"drive/MyDrive/LaneDetection/LaneViT_regression_state_dict_2.pt") #Save the parameters also. Remember vgg16 parameters are seperately needed here

#### Training all together : Experimental

In [ ]:
class LaneLoss(nn.Module):
    def __init__(self):
        super(LaneLoss, self).__init__()
        self.regression_loss_fn = nn.MSELoss(reduction='none')  # Use 'none' to calculate loss per sample
        self.confidence_loss_fn = nn.BCEWithLogitsLoss()  # Applies a logistic loss to the confidence scores

    def forward(self, regression_pred, regression_target, confidence_pred, confidence_target):
        # Regression loss where confidence is 1
        regression_pred = regression_pred.view(-1, 19, 17)
        regression_target = regression_target.view(-1, 19, 17)

        #print(regression_pred.shape)

        # Apply confidence target to each column of the regression output
        confidence_target_expanded = confidence_target.unsqueeze(1).expand_as(regression_pred)
        #print(confidence_target_expanded)
        regression_loss = self.regression_loss_fn(regression_pred, regression_target)  # Shape: [batch_size, 19, 17]
        regression_loss = (regression_loss * confidence_target_expanded).mean()

        # Confidence loss (logistic loss)
        confidence_loss = self.confidence_loss_fn(confidence_pred, confidence_target)

        # Combine the losses
        total_loss = regression_loss/10 + confidence_loss #Dividing by 10 is experimental

        return total_loss

In [ ]:
model = torch.load("drive/MyDrive/LaneDetection/LaneViT_regression.pt")

In [ ]:
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
loss_fn = LaneLoss()
lane_dataset = LaneDataset(data_line_map=data_line_map,data_stline_map=data_stline_map,scale=(224,224))
dataloader = DataLoader(lane_dataset, batch_size=32,shuffle=True)
# Initialize the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10  # Adjust the number of epochs as needed



In [ ]:
for param in model.vggblock.parameters():
            param.requires_grad = False #Disable the vgg trainings

for param in model.upsample_neck.parameters():
            param.requires_grad = False #Neck is already trained

for param in model.laneconfidence_head.parameters():
            param.requires_grad = False #Confidence is already trained

for param in model.regression_head.parameters():
            param.requires_grad = True #Now train only for regression head


In [ ]:
import time

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    #print("Start",time.time())
    for batch_idx, (images, regression_targets, confidence_targets) in enumerate(dataloader):
        images = images.to(device)
        # max_lane_dis = max(max_lane_dis,regression_targets.max())
        # min_lane_dis = min(min_lane_dis,regression_targets.min())
        #print("Data fetched",time.time()
        regression_targets = regression_targets.to(device)
        confidence_targets = confidence_targets.to(device)
        #print("Datafetched",time.time())
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass

        confidence_pred,regression_pred = model(images)

        # Calculate loss
        loss = loss_fn(regression_pred, regression_targets,confidence_pred,confidence_targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Update running loss
        running_loss += loss.item()

        # Print progress
        if batch_idx % 20 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}')
            torch.save(model,"drive/MyDrive/LaneDetection/model_reg.pt")
        #print("Model stepped",time.time())
    # Print epoch loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

print("Training completed.")